# Testing SentenceBERT for semantic similarity

* https://medium.com/analytics-vidhya/semantic-similarity-in-sentences-and-bert-e8d34f5a4677
* https://towardsdatascience.com/word-embedding-using-bert-in-python-dd5a86c00342
* https://github.com/huggingface/transformers

Install hugginface transformers and sentence-transformers

In [ ]:
!pip install transformers # https://github.com/huggingface/transformers
!pip install -U sentence-transformers # https://github.com/UKPLab/sentence-transformers

In [2]:
import pandas as pd
import numpy as np
import torch

from sentence_transformers import SentenceTransformer
from scipy.spatial import distance
import nltk

model = SentenceTransformer('bert-large-nli-stsb-mean-tokens') # Load the BERT model. Semantic Textual Similarity models are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

100%|██████████| 1.24G/1.24G [01:05<00:00, 19.0MB/s]


## 1. Load the sts-benchmark data and remove lines that contain errors.

In [ ]:
train_df = pd.pandas.read_table(
    'stsbenchmark/sts-train.csv',
    error_bad_lines=False,
    skip_blank_lines=True,
    usecols=[4, 5, 6],
    names=["score", "s1", "s2"])

## 2. A quick look at the dataset we are using

In [ ]:
train_df.head()

,score,s1,s2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


In [ ]:
train_df.tail()

,score,s1,s2
5706,0.0,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia
5707,0.0,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...
5708,0.0,President heading to Bahrain,President Xi: China to continue help to fight ...
5709,0.0,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders
5710,0.0,Putin spokesman: Doping charges appear unfounded,The Latest on Severe Weather: 1 Dead in Texas ...


## 3. Comparing two sentence paires with SentenceBert as an example

In [ ]:
s1 = train_df.loc[0][1]
s2 = train_df.loc[0][2]
s3 = train_df.loc[45][1]
s4 = train_df.loc[45][2]

print(f's1 = {s1}')
print(f's2 = {s2}')
print('\n')
print(f's3 = {s3}')
print(f's4 = {s4}')

s1 = A plane is taking off.
s2 = An air plane is taking off.


s3 = A man is playing the piano.
s4 = A woman is playing the violin.


In [ ]:
s1_embedding = model.encode(s1)
s2_embedding = model.encode(s2)
s3_embedding = model.encode(s3)
s4_embedding = model.encode(s4)

print(f's1 vs s2 = {distance.cosine(s1_embedding,s2_embedding)}')
print(f'Human score = {train_df.loc[0][0]}')
print(f'SentenceBERT Score = {round((1-distance.cosine(s1_embedding,s2_embedding))*5,1)}')

print(f's3 vs s4 = {distance.cosine(s3_embedding,s4_embedding)}')
print(f'Human score = {train_df.loc[45][0]}')
print(f'SentenceBERT Score = {round((1-distance.cosine(s3_embedding,s4_embedding))*5,1)}')

print(f's1 vs s3 = {distance.cosine(s1_embedding,s3_embedding)}')
print(f's1 vs s4 = {distance.cosine(s1_embedding,s4_embedding)}')

s1 vs s2 = 0.017929434776306152
Human score = 5.0
SentenceBERT Score = 4.9
s3 vs s4 = 0.7746940553188324
Human score = 1.0
SentenceBERT Score = 1.1
s1 vs s3 = 0.8804589882493019
s1 vs s4 = 0.8903428316116333


## 4. Getting the human scores and the SentenceBERT scores and comparing them

### 4.1 Load the data and preprocess it

In [ ]:
dev_df = pd.pandas.read_table(
    'stsbenchmark/sts-dev.csv',
    error_bad_lines=False,
    skip_blank_lines=True,
    usecols=[4, 5, 6],
    names=["score", "s1", "s2"])

# removes punctuation from sentences
tokenizer = nltk.RegexpTokenizer(r"\w+")

# For some reason some of the sentences were "float" datatypes 
dev_df['s1'] = dev_df['s1'].astype(str)
dev_df['s2'] = dev_df['s2'].astype(str)


dev_df['s1'] = dev_df.apply(lambda row: tokenizer.tokenize(row['s1']), axis=1)
dev_df['s1'] = dev_df.apply(lambda row: ' '.join(row['s1']).lower() , axis=1)

dev_df['s2'] = dev_df.apply(lambda row: tokenizer.tokenize(row['s2']), axis=1)
dev_df['s2'] = dev_df.apply(lambda row: ' '.join(row['s2']).lower() , axis=1)

In [ ]:
dev_df.head()

,score,s1,s2
0,5.00,a man with a hard hat is dancing,a man wearing a hard hat is dancing
1,4.75,a young child is riding a horse,a child is riding a horse
2,5.00,a man is feeding a mouse to a snake,the man is feeding a mouse to the snake
3,2.40,a woman is playing the guitar,a man is playing guitar
4,2.75,a woman is playing the flute,a man is playing a flute


### 4.2 Get the scores and normalize them

In [ ]:
dev_scores = dev_df['score'].tolist()

score_human = []

for row in dev_scores:
    score = row/5
    score_human.append(score)

In [ ]:
score_machine = []

for row in dev_df.itertuples(index=False):
    s1_embedding = model.encode(str(row[1]))
    s2_embedding = model.encode(str(row[2]))
    score = (1-distance.cosine(s1_embedding,s2_embedding))
    score_machine.append(score)

### 4.3 Compare human and fastText scores

In [ ]:
from scipy.stats import pearsonr

result, _ = pearsonr(score_machine, score_human)
print('Pearsonr:', end=' ')
print("%.1f" % (result*100))

Pearsonr: 87.4
